In [38]:
import numpy as np
import scipy.stats as stats
from math import log, sqrt, exp
from scipy.stats import norm
import sys
sys.path.append('/Users/ansel_li/Fintech545/public/')
from risk_mgmt import black_s

In [21]:
# Input parameters
S = 165  # Current stock price
K = 165  # Strike price
t = (np.datetime64('2022-03-13') - np.datetime64('2022-03-13')).astype(int) / 365.0
T = (np.datetime64('2022-04-15') - np.datetime64('2022-03-13')).astype(int) / 365.0
r = 0.0425  # Risk-free interest rate
q = 0.0053  # Continuously compounding dividend yield
sigma = 0.20  # Implied volatility


In [22]:
def gbsm_greeks(S, K, t, T, r, q, sigma, option_type='call'):
    tau = T - t
    d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
    d2 = d1 - sigma * sqrt(tau)

    N_d1 = stats.norm.cdf(d1)
    N_d2 = stats.norm.cdf(d2)

    if option_type == 'call':
        delta = exp(-q * tau) * N_d1
        gamma = exp(-q * tau) * N_d1 / (S * sigma * sqrt(tau))
        vega = S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)
        theta = -S * np.exp(q * tau) * N_d1 * sigma / (2 * np.sqrt(tau)) - r * K * np.exp(-r * tau) * -N_d2 + q * S * np.exp(-q * tau) * -N_d1
        rho = K * tau * np.exp(-r * tau) * norm.cdf(d2)
    elif option_type == 'put':
        delta = -exp(-q * tau) * (1 - N_d1)
        gamma = exp(-q * tau) * N_d1 / (S * sigma * sqrt(tau))
        vega = S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)
        theta = -S * np.exp(q * tau) * N_d1 * sigma / (2 * np.sqrt(tau)) + r * K * np.exp(-r * tau) * -N_d2 - q * S * np.exp(-q * tau) * -N_d1
        rho = -K * tau * np.exp(-r * tau) * norm.cdf(-d2)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return delta, gamma, vega, theta, rho

In [23]:
call_greeks = gbsm_greeks(S, K, t, T, r, q, sigma, option_type='call')
put_greeks = gbsm_greeks(S, K, t, T, r, q, sigma, option_type='put')

print("Call Greeks: Delta, Gamma, Vega, Theta, Rho")
print(call_greeks)
print("Put Greeks: Delta, Gamma, Vega, Theta, Rho")
print(put_greeks)

Call Greeks: Delta, Gamma, Vega, Theta, Rho
(0.5340091224850149, 0.053817584150433175, 19.710179716477544, -26.233902394104227, 7.583586080244792)
Put Greeks: Delta, Gamma, Vega, Theta, Rho
(-0.4655118142202754, 0.053817584150433175, 19.710179716477544, -32.429640382502015, -7.277010958127815)


In [24]:
def finite_difference(S, K, t, T, r, q, sigma, option_type='call', delta_shift=0.01):
    S_up = S + S * delta_shift
    S_down = S - S * delta_shift
    sigma_up = sigma + delta_shift
    sigma_down = sigma - delta_shift
    r_up = r + delta_shift
    r_down = r - delta_shift

    def option_price(S, K, t, T, r, q, sigma, option_type):
        tau = T - t
        d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
        d2 = d1 - sigma * sqrt(tau)

        if option_type == 'call':
            return S * exp(-q * tau) * stats.norm.cdf(d1) - K * exp(-r * tau) * stats.norm.cdf(d2)
        elif option_type == 'put':
            return K * exp(-r * tau) * stats.norm.cdf(-d2) - S * exp(-q * tau) * stats.norm.cdf(-d1)

    # Finite difference calculations
    delta_fd = (option_price(S_up, K, t, T, r, q, sigma, option_type) - option_price(S_down, K, t, T, r, q, sigma, option_type)) / (2 * S * delta_shift)
    gamma_fd = (option_price(S_up, K, t, T, r, q, sigma, option_type) - 2 * option_price(S, K, t, T, r, q, sigma, option_type) + option_price(S_down, K, t, T, r, q, sigma, option_type)) / (S * delta_shift)**2
    vega_fd = (option_price(S, K, t, T, r, q, sigma_up, option_type) - option_price(S, K, t, T, r, q, sigma_down, option_type)) / (2 * 0.01)
    theta_fd = - (option_price(S, K, t + 1/365, T, r, q, sigma, option_type) - option_price(S, K, t, T, r, q, sigma, option_type)) / (-1/365)
    rho_fd = (option_price(S, K, t, T, r_up, q, sigma, option_type) - option_price(S, K, t, T, r_down, q, sigma, option_type)) / (2 * 0.01)

    return delta_fd, gamma_fd, vega_fd, theta_fd, rho_fd

In [25]:
call_greeks_fd = finite_difference(S, K, t, T, r, q, sigma, option_type='call')
put_greeks_fd = finite_difference(S, K, t, T, r, q, sigma, option_type='put')

print("Call Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho")
print(call_greeks_fd)
print("Put Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho")
print(put_greeks_fd)

import pandas as pd

# Combine the results in a dictionary
greek_data = {
    "Call Closed-form": call_greeks,
    "Call Finite Diff.": call_greeks_fd,
    "Put Closed-form": put_greeks,
    "Put Finite Diff.": put_greeks_fd
}

# Create a DataFrame
greeks_comparison = pd.DataFrame(greek_data, index=["Delta", "Gamma", "Vega", "Theta", "Rho"])

# Display the DataFrame
print(greeks_comparison)

Call Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho
(0.5337431087559965, 0.039948679750147466, 19.71009507625965, -25.06750653518303, 7.583554488655153)
Put Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho
(-0.4657778279492933, 0.03994867975014225, 19.71009507625965, -18.955580817535633, -7.277044574261282)
       Call Closed-form  Call Finite Diff.  Put Closed-form  Put Finite Diff.
Delta          0.534009           0.533743        -0.465512         -0.465778
Gamma          0.053818           0.039949         0.053818          0.039949
Vega          19.710180          19.710095        19.710180         19.710095
Theta        -26.233902         -25.067507       -32.429640        -18.955581
Rho            7.583586           7.583554        -7.277011         -7.277045


In [26]:
def binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=None, div_amt=0):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r) * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    # Create the stock price tree
    stock = np.zeros((n + 1, n + 1))
    stock[0, 0] = S

    for i in range(1, n + 1):
        stock[i, 0] = stock[i - 1, 0] * u
        for j in range(1, i + 1):
            stock[i, j] = stock[i - 1, j - 1] * d

    # Adjust stock prices for dividends
    if div_date is not None and exercise_type == 'american':
        div_steps = int((div_date - np.datetime64('2022-03-13')).astype(int) / 365.0 * n)
        stock[div_steps:] -= div_amt

    # Create the option price tree
    option = np.zeros_like(stock)

    # Calculate the option's value at maturity
    if option_type == 'call':
        option[:, -1] = np.maximum(stock[:, -1] - K, 0)
    elif option_type == 'put':
        option[:, -1] = np.maximum(K - stock[:, -1], 0)

    # Step back through the tree to calculate the option's value at each node
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option[i, j] = discount * (p * option[i + 1, j] + (1 - p) * option[i + 1, j + 1])

            # Calculate the option's value if exercised early
            if exercise_type == 'american':
                if option_type == 'call':
                    early_exercise = np.maximum(stock[i, j] - K, 0)
                elif option_type == 'put':
                    early_exercise = np.maximum(K - stock[i, j], 0)

                option[i, j] = np.maximum(option[i, j], early_exercise)

    return option




def binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=None, div_amt=0, delta_shift=0.01, rate_shift=0.0001, vol_shift=0.01):
    option_tree = binomial_tree(S, K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_up = binomial_tree(S * (1 + delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_down = binomial_tree(S * (1 - delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_up = binomial_tree(S, K, T, r, sigma + vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_down = binomial_tree(S, K, T, r, sigma - vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_up = binomial_tree(S, K, T, r + rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_down = binomial_tree(S, K, T, r - rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)

    # Calculate Greeks
    delta = (option_tree_S_up[0, 0] - option_tree_S_down[0, 0]) / (2 * S * delta_shift)
    gamma = (option_tree_S_up[0, 0] - 2 * option_tree[0, 0] + option_tree_S_down[0, 0])/ (S * delta_shift) ** 2
    theta = -(option_tree[0, 0] - binomial_tree(S, K, T - 1/n, r, sigma, n, option_type, exercise_type, div_date, div_amt)[0, 0]) / (1/n)

    vega = (option_tree_vol_up[0, 0] - option_tree_vol_down[0, 0]) / (2 * vol_shift)
    rho = (option_tree_rate_up[0, 0] - option_tree_rate_down[0, 0]) / (2 * rate_shift)

    return delta, gamma, theta, vega, rho



In [27]:
n=100
div_date = np.datetime64('2022-04-11')
div_amt = 0.88  # Dividend amount

call_without_div = binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american')[0, 0]
put_without_div = binomial_tree(S, K, T, r, sigma, n, option_type='put', exercise_type='american')[0, 0]
call_with_div = binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=div_date, div_amt=div_amt)[0, 0]
put_with_div = binomial_tree(S, K, T, r, sigma, n, option_type='put', exercise_type='american', div_date=div_date, div_amt=div_amt)[0, 0]

print("American Call Option without Dividends:", call_without_div)
print("American Put Option without Dividends:", put_without_div)
print("American Call Option with Dividends:", call_with_div)
print("American Put Option with Dividends:", put_with_div)

American Call Option without Dividends: 4.261677061030071
American Put Option without Dividends: 3.680829925622103
American Call Option with Dividends: 3.811548561304113
American Put Option with Dividends: 4.11598660293646


In [28]:
# calculate greeks
call_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american')
put_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american')
call_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=div_date, div_amt=div_amt)
put_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american', div_date=div_date, div_amt=div_amt)

# create dataframe for greeks
greek_labels = ['Delta', 'Gamma', 'Theta', 'Vega', 'Rho']
call_greeks = pd.DataFrame({'Call without div': call_greeks_without_div, 'Call with div': call_greeks_with_div}, index=greek_labels)
put_greeks = pd.DataFrame({'Put without div': put_greeks_without_div, 'Put with div': put_greeks_with_div}, index=greek_labels)

# display greeks dataframes
print('Call Option Greeks:')
print(call_greeks)

print('\nPut Option Greeks:')
print(put_greeks)

Call Option Greeks:
       Call without div  Call with div
Delta          0.536898       0.500120
Gamma          0.038292       0.045113
Theta        -25.932238     -25.884008
Vega          19.657366      19.821456
Rho            7.550520       7.066599

Put Option Greeks:
       Put without div  Put with div
Delta        -0.471247     -0.508301
Gamma         0.039587      0.046210
Theta       -19.577328    -19.518492
Vega         19.621387     19.702896
Rho          -5.868226     -6.173442


In [29]:
# Calculate the sensitivity of the call and put to a change in dividend amount
Delta_call_dividend = (call_with_div - call_without_div) / div_amt
Delta_put_dividend = (put_with_div - put_without_div) / div_amt

print("Delta Call with Dividend: ", Delta_call_dividend)
print("Delta Put with Dividend: ", Delta_put_dividend)

Delta Call with Dividend:  -0.511509658779498
Delta Put with Dividend:  0.49449622422086037


# Problem 2

In [30]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from risk_mgmt import simulation, VaR, calc_return
# Load the data from CSV files
portfolio = pd.read_csv('problem2.csv')
prices = pd.read_csv('DailyPrices.csv')



# Set the date and other parameters
dividend = 1
rf = 0.0425
q = 0.0053
current_date = '2023-03-03'
current_date = pd.to_datetime(current_date)
S = 151.03


In [31]:
# call the function that add implied vol and T to the portfolio.csv
portfolio = black_s.add_VolnT(S,q,rf, portfolio, current_date)
portfolio.head()

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,T,Implied_Volatility
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.267740
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.134247,0.259735
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.267740
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85,0.134247,0.259735
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.267740


In [32]:
# Filter the historical prices for AAPL
return_aapl = simulation.log_returns(prices['AAPL'])
m_return = np.mean(return_aapl)
return_aapl = return_aapl - m_return
mu, std = norm.fit(return_aapl)
n_simulations = 10000
n_days = 10
simulated_returns = np.random.normal(loc=mu, scale=std, size=(n_simulations, n_days))


In [33]:
initial_price = 151.03

# Calculate the price series
price_series = pd.DataFrame(initial_price * np.exp(simulated_returns.cumsum(axis=1)))
simulation_price = price_series.iloc[:,9]

In [34]:
simulated_returns[:,-1]

array([ 0.00447394, -0.01284723, -0.01743315, ..., -0.00697701,
        0.05573792, -0.0223358 ])

In [35]:
# Get a list of unique portfolio names
portfolios = portfolio['Portfolio'].unique()

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=portfolios, index=['Mean', 'VaR', 'ES'])

# Iterate through each portfolio and calculate the Mean, VaR, and ES
for portname in portfolios:
    portfolio_data = portfolio[portfolio['Portfolio'] == portname]
    simulate_portfolio_values = pd.DataFrame(black_s.portfolio_value(portfolio_data, simulation_price, rf, q, current_date, day_ahead=10))
    current_portfolio_values = pd.DataFrame(black_s.portfolio_value(portfolio_data, [initial_price], rf, q, current_date))

    # Calculate the difference between simulated and current portfolio values
    current_portfolio_value = current_portfolio_values.values[0][0]
    diff_portfolio_values = simulate_portfolio_values.apply(lambda x: x - current_portfolio_value)

    #print(diff_portfolio_values.values[0])
    # Calculate the Mean, VaR, and ES for the difference
    Mean = diff_portfolio_values.mean().values[0]

    var = VaR.calculate_var(diff_portfolio_values.values)
    es = VaR.calculate_es(diff_portfolio_values.values)

    # Add the results to the DataFrame
    results_df.at['Mean', portname] = Mean
    results_df.at['VaR', portname] = var
    results_df.at['ES', portname] = es

# Display the results
results_df.T





,Mean,VaR,ES
Straddle,1.789586,1.376698,1.386685
SynLong,-0.025505,17.264889,20.881793
CallSpread,-0.099052,4.001082,4.251238
PutSpread,0.34023,2.684263,2.833844
Stock,0.180201,17.015232,20.611717
Call,0.88204,6.163967,6.439931
Put,0.907546,4.438271,4.630509
CoveredCall,-0.840243,13.15096,16.655005
ProtectedPut,0.949867,8.297707,8.829118


In [36]:
def delta(S, K, T, r, q, sigma, option_type):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    if option_type == 'Call':
        delta = np.exp(-q * T) * norm.cdf(d1)
    elif option_type == 'Put':
        delta = np.exp(-q * T) * (norm.cdf(d1) - 1)

    return delta

def delta_normal_var_pnl(portfolio, initial_price, rf, q, current_date, simulated_prices):
    portfolio_delta = 0

    for _, row in portfolio.iterrows():
        holding = row['Holding']

        if row['Type'] == 'Stock':
            portfolio_delta += holding
        elif row['Type'] == 'Option':
            T = ((row['ExpirationDate'] - current_date) / pd.Timedelta(1, 'D')) / 365
            sigma = row['Implied_Volatility']
            option_delta = delta(initial_price, row['Strike'], T, rf, q, sigma, row['OptionType'])
            portfolio_delta += holding * option_delta

    simulated_pnl = (simulated_prices - initial_price) * portfolio_delta
    dn_var = np.percentile(simulated_pnl, 5)
    return dn_var, simulated_pnl


results_df1 = pd.DataFrame(columns=portfolios, index=['VaR', 'ES'])
# Calculate Delta-Normal VaR and ES for each portfolio
for portname in portfolios:
    portfolio_data = portfolio[portfolio['Portfolio'] == portname]
    dn_var, pnl = delta_normal_var_pnl(portfolio_data, initial_price, rf, q, current_date, simulation_price)
    dn_es = np.mean(pnl[pnl <= dn_var])

    results_df1.at['VaR', portname] = -dn_var
    results_df1.at['ES', portname] = -dn_es
results_df1.T

,VaR,ES
Straddle,2.306497,2.794018
SynLong,16.988073,20.578817
CallSpread,4.803094,5.818317
PutSpread,4.237225,5.437323
Stock,17.015232,20.611717
Call,9.647285,11.686418
Put,7.942763,10.19237
CoveredCall,9.771212,11.836539
ProtectedPut,11.862294,14.36961


In [37]:
import pandas as pd
import statsmodels.api as sm

# Load daily price data
prices_df = pd.read_csv('DailyPrices.csv', index_col=0)

# Load Fama French 3 factor return time series data
ff3_df = pd.read_csv('F-F_Research_Data_Factors_daily.csv', index_col=0)
ff3_df.index.name = 'Date'

ff3_df = ff3_df[['Mkt-RF', 'SMB', 'HML', 'RF']]

# Load Carhart Momentum time series data
mom_df = pd.read_csv('F-F_Momentum_Factor_daily.csv', index_col=0, parse_dates=True)
mom_df = mom_df[['Mom']]

# Merge daily prices, Fama French 3 factor and Carhart Momentum data
df = prices_df.join(ff3_df, how='inner').join(mom_df, how='inner')
print(ff3_df)
# Define the stocks we want to fit the model for
stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE', 'AMZN', 'BRK-B', 'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']

# Create an empty DataFrame to store results
results_df = pd.DataFrame(columns=['Stock', 'alpha', 'beta_mkt', 'beta_smb', 'beta_hml', 'beta_mom'])

# Loop through the stocks and fit the 4 factor model for each
for stock in stocks:
    # Define the dependent and independent variables
    y = df[stock] - df['RF']
    
    X = sm.add_constant(df[['Mkt-RF', 'SMB', 'HML', 'Mom']])
    # Fit the 4 factor model
    model = sm.OLS(y, X)
    results = model.fit()
    # Store the results in the results DataFrame
    results_df.loc[len(results_df)] = [stock, results.params['const'], results.params['Mkt-RF'], results.params['SMB'], results.params['HML'], results.params['Mom']]

# Print the results
print(results_df)


          Mkt-RF   SMB   HML     RF
Date                               
19260701    0.10 -0.25 -0.27  0.009
19260702    0.45 -0.33 -0.06  0.009
19260706    0.17  0.30 -0.39  0.009
19260707    0.09 -0.58  0.02  0.009
19260708    0.21 -0.38  0.19  0.009
...          ...   ...   ...    ...
20230125    0.00 -0.04  0.65  0.017
20230126    1.08 -0.58  0.01  0.017
20230127    0.36  0.62 -1.16  0.017
20230130   -1.38 -0.10  0.72  0.017
20230131    1.57  0.99 -0.06  0.017

[25419 rows x 4 columns]


ValueError: zero-size array to reduction operation maximum which has no identity

In [39]:
from datetime import datetime

In [40]:
# Load the datasets
fama_french = pd.read_csv('F-F_Research_Data_Factors_daily.csv')
momentum = pd.read_csv('F-F_Momentum_Factor_daily.csv')
prices = pd.read_csv('DailyPrices.csv')

# Filter the datasets for the past 10 years
current_year = datetime.now().year
start_date = (current_year - 10) * 10000 + 101
end_date = (current_year - 1) * 10000 + 1231

fama_french = fama_french[(fama_french['Date'] >= start_date) & (fama_french['Date'] <= end_date)]
momentum = momentum[(momentum['Date'] >= start_date) & (momentum['Date'] <= end_date)]
# Preprocessing
fama_french['Date'] = pd.to_datetime(fama_french['Date'], format='%Y%m%d')
momentum['Date'] = pd.to_datetime(momentum['Date'], format='%Y%m%d')
prices['Date'] = pd.to_datetime(prices['Date'])

# Calculate stock returns
chosen_stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE', 'AMZN', 'BRK-B', 'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']
stock_prices = prices[chosen_stocks]

# Calculate stock returns
stock_returns = calc_return.return_calculate(prices).set_index('Date')
select_stock = stock_returns[chosen_stocks]

# Align the data
fama_french = fama_french.set_index('Date')
momentum = momentum.set_index('Date')

# Merge the stock returns, Fama-French factors, and momentum factors
merged_data = pd.concat([select_stock, fama_french, momentum], axis=1).dropna()

fama_french = merged_data[['Mkt-RF', 'SMB', 'HML', 'RF']]
momentum = merged_data[['Mom']]

# Subtract the risk-free rate from stock returns
stock_returns = select_stock.sub(fama_french['RF'], axis=0).dropna()


/Users/ansel_li/Fintech545/public/risk_mgmt/calc_return.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:, i]


In [41]:
stock_returns

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2022-02-15,0.023152,0.015158,0.008073,0.019724,0.018542,0.091812,0.004836,-0.000201,0.008658,0.006109,0.000510,-0.012535,0.053291,0.014624,0.011006,0.025655,0.007987,0.010326,0.007803,0.020496
2022-02-16,-0.001389,-0.020181,0.003806,0.003643,-0.001167,0.000604,-0.008974,-0.002209,0.010159,-0.001739,0.007588,0.004616,0.001041,0.001810,0.004389,0.010535,0.008268,-0.000598,-0.002302,-0.000369
2022-02-17,-0.021269,-0.040778,-0.020227,-0.024103,-0.029282,-0.075591,-0.006141,-0.015700,-0.021809,-0.006653,0.011455,-0.001531,-0.050943,-0.023032,-0.018399,-0.021746,-0.037746,-0.006100,-0.033767,0.028018
2022-02-18,-0.009356,-0.007462,-0.005379,-0.010035,-0.009631,-0.035296,-0.003075,-0.007566,-0.013262,0.003987,0.000501,-0.011121,-0.022103,0.004689,-0.008548,-0.010396,-0.016116,-0.010719,-0.002388,0.025820
2022-02-22,-0.017812,-0.019790,-0.011329,-0.004487,-0.000729,-0.010659,-0.088506,-0.020606,-0.015753,-0.002033,-0.012320,-0.011634,-0.041366,-0.001775,-0.006152,-0.021604,-0.004521,-0.013590,-0.008703,-0.015906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,-0.018798,-0.008145,-0.007994,-0.009910,-0.013733,-0.024671,-0.007744,-0.012321,0.001425,-0.003446,-0.013175,0.010445,-0.033551,-0.011255,-0.012245,-0.000539,0.000750,-0.013458,-0.013530,-0.012619
2022-12-27,-0.029878,-0.025827,-0.014720,-0.008549,-0.023414,-0.087353,-0.013427,-0.029506,-0.041924,-0.019067,-0.007286,-0.002106,-0.130089,-0.012496,-0.013765,-0.034634,-0.036621,-0.016282,-0.014152,-0.014947
2022-12-28,-0.046685,-0.026780,-0.022654,-0.028162,-0.026255,-0.022019,-0.027954,-0.022454,-0.030692,-0.022938,-0.028926,-0.032426,0.017089,-0.010535,-0.022302,-0.041472,-0.031677,-0.020340,-0.008622,-0.025678


In [42]:
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
X = merged_data[factors]
X = sm.add_constant(X)

betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])
for stock in chosen_stocks:
    model = sm.OLS(stock_returns[stock], X).fit()
    betas.loc[stock] = model.params[factors]
    alphas.loc[stock] = model.params['const']

In [43]:
alphas.head()

,Alpha
AAPL,-0.006524
META,-0.006976
UNH,-0.005606
MA,-0.005924
MSFT,-0.006178


In [44]:
betas.head()

,Mkt-RF,SMB,HML,Mom
AAPL,0.012104,-0.004594,-0.004698,0.001483
META,0.011547,-0.004748,-0.003464,-0.006965
UNH,0.007982,-0.003855,-0.002904,0.005874
MA,0.01072,-0.001046,0.000448,-0.000116
MSFT,0.011336,-0.007548,-0.005798,0.000479


In [73]:
# Calculate estimated annual return for each stock
est_annual_return = pd.DataFrame((alphas['Alpha']*252 + \
                    betas['Mkt-RF']*merged_data['Mkt-RF'].mean()*252 + \
                    betas['SMB']*merged_data['SMB'].mean()*252 + \
                    betas['HML']*merged_data['HML'].mean()*252 + \
                    betas['Mom']*merged_data['Mom'].mean()*252 + \
                    fama_french['RF'].mean()*252))

In [74]:
est_annual_return

,0
AAPL,-0.227886
META,-0.485602
UNH,0.171017
MA,-0.027627
MSFT,-0.161588
NVDA,-0.373268
HD,-0.043308
PFE,0.095651
AMZN,-0.577929
BRK-B,0.006792


In [75]:
cov_m = select_stock[stocks].cov() * 252

In [93]:
from scipy.optimize import minimize

def max_sharpe_ratio_weights(cov_m, exp_returns, rf):
    num_stocks = len(exp_returns)
    
    # Define the Sharpe Ratio objective function to be minimized
    def neg_sharpe_ratio(weights):
        port_return = np.dot(weights, exp_returns)
        port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_m, weights)))
        sharpe_ratio = (port_return - rf) / port_volatility
        return -sharpe_ratio
    
    # Define the constraints
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # The sum of the weights must be 1
    bounds = tuple([(0, 1) for i in range(num_stocks)]) # The weights must be between 0 and 1
    
    # Find the portfolio weights that maximize the Sharpe Ratio
    initial_weights = np.ones(num_stocks) / num_stocks # Start with equal weights
    opt_results = minimize(neg_sharpe_ratio, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return opt_results.x.round(4), opt_results.fun 


In [97]:
weights, sharpe = max_sharpe_ratio_weights(cov_m, est_annual_return, 0.0425)
print("Sharpe ratio is:", -sharpe)

Sharpe ratio is: 1.2851485046659787


In [98]:
stock_names = cov_m.columns.values
portfolio_weights = pd.DataFrame({'Stock': stock_names, 'Weight': weights})
portfolio_weights.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Stock,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Weight,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.676,0.0,0.0,0.0,0.0,0.0,0.324,0.0,0.0
